In [1]:
import pandas as pd
from sklearn.preprocessing import normalize
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
import sys
import os
import re
from nltk.corpus import stopwords

import spacy
import en_core_web_sm


In [3]:
df = pd.read_csv("train.csv")
##just checking the dataframe again
df.value_counts(df["is_duplicate"]==1)

is_duplicate
False    255027
True     149263
dtype: int64

In [4]:
#encosing questions

df["question1"] = df["question1"].apply(lambda x : str(x))
df["question2"] = df["question2"].apply(lambda x : str(x))


In [5]:
# merging text
questions = list(df["question1"] + df["question2"])
questions[0]

'What is the step by step guide to invest in share market in india?What is the step by step guide to invest in share market?'

In [6]:
tfidf = TfidfVectorizer(lowercase = False)
tfidf.fit_transform(questions)


<404290x110542 sparse matrix of type '<class 'numpy.float64'>'
	with 6336617 stored elements in Compressed Sparse Row format>

In [7]:
word2tfidf = dict(zip(tfidf.get_feature_names() , tfidf.idf_))


In [19]:
# make sure to write this code before importing en_core_web_sm before module
!python -m spacy download en

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=78bd3fba2e9000ff26514c8dfbd5a37a45db7ef6d6a59c27b4b4b3d306355ea8
  Stored in directory: C:\Users\Acer\AppData\Local\Temp\pip-ephem-wheel-cache-7tr97bk3\wheels\ee\4d\f7\563214122be1540b5f9197b52cb3ddb9c4a8070808b22d5a84
Successfully built en-core-web-sm
symbolic link created for C:\Users\Acer\anaconda3\lib\site-packages\spacy\data\en <<===>> C:\Users\Acer\anaconda3\lib\site-packages\en_core_web_sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[+] Linking successful
C:\Users\Acer\anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\Acer\anaconda3\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


In [2]:
nlp = en_core_web_sm.load()


In [36]:
vecs1 = []
for qu1 in tqdm(list(df["question1"])):
    doc1 = nlp(qu1)
    mean_vec1 = np.zeros([len(doc1) , len(doc1[0].vector)])
    
    for word1 in doc1:
        #word2vec
        vec1 = word1.vector
        #fetch df scores
        
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf =0
        #compute final vec
        mean_vec1 += vec1*idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df["q1_feats_m"] = list(vecs1)
    

    


100%|██████████| 404290/404290 [41:47<00:00, 161.24it/s] 


In [16]:
vecs2 = []
for qu2 in tqdm(list(df["question2"])):
    doc2 = nlp(qu2)
    mean_vec2 = np.zeros([len(doc2) , len(doc2[0].vector)])
    
    for word2 in doc2:
        vec2 = word2.vector
        
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
            
            mean_vec2 += vec2*idf
    mean_vec2 = mean_vec2.mean(axis = 0)
    vecs2.append(mean_vec2)
df["q2_feats_m"] = list(vecs2)


100%|██████████| 404290/404290 [52:42<00:00, 127.83it/s] 


In [21]:
dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
dfppro = pd.read_csv("df_fe_without_preprocessing",encoding='latin-1')

In [22]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [ ]:
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [ ]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')